### Note.

このファイルだけでは動作しないため  
同じディレクトリに `chatbot_system_role.md` を配備してください

In [ ]:
### =======================================
### ライブラリについて
### =======================================

# builtin modules
import os                 # ... 環境変数をロードするために使います
import re                 # ... 正規表現を利用して文字列をチェックするために使います
from pathlib import Path  # ... (抽象的で)高度な Path 周りの関数を扱うために使います
import traceback          # ... 主に例外のトレースバックを出力するデバッグ用途に用います

# 3rd party modules
from openai import OpenAI
# from dotenv import load_dotenv


### =======================================
### メインの実装部分
### =======================================

MODEL_NAME = "gpt-4o-mini"
DEBUG_FLAG = False         # ... True にすると Debug モードになります、ログを出力します
KEEP_MESSAGE_LIMIT = 3     # ... 直近過去何往復のやりとりまで記憶して送信するかです
SYSTEM_ROLE_MESSAGE_PATH = Path(".") / "chatbot_system_role.md" 
                           # ... システムロールの指示を記載したファイルです

def main():
    # API キーを自動的にロードして OpenAI API クライアントを作る
    client = OpenAIClientGenerator.generate(display_debug_messages=DEBUG_FLAG)

    # ロギング用の関数 (デバッグ用に使います)
    log = OpenAIClientGenerator.get_logger(DEBUG_FLAG)

    # システム Role 用のメッセージ
    # chatbot_system_role.md をロードします
    with open(SYSTEM_ROLE_MESSAGE_PATH, "r") as f:
        system_role_content = f.read().strip()
    system_role_message = {
        "role": "system",
        "content": system_role_content
    }

    # メッセージを格納するリスト
    user_messages = []

    log("Started main program.")
    while(True):
        # ユーザーからの質問を受付
        message = input("メッセージを入力:")

        # 質問が入力されなければ終了
        if message.strip() == "": break
        print(f"質問:{message}")

        # メッセージにユーザーからの質問を追加
        user_messages.append({"role": "user", "content": message.strip()})

        # やりとりが 2 * KEEP_MESSAGE_LIMIT を超えたら古いメッセージから削除
        if len(user_messages) > (2 * KEEP_MESSAGE_LIMIT):
            del_message = user_messages.pop(0)
            log(f"Deleted message. { del_message }")

        # APIへリクエスト
        messages = [ system_role_message ] + user_messages
        log(f"Requesting API. model={MODEL_NAME} {messages=}")
        stream = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            stream=True,
        )

        # 言語モデルからの回答を表示
        print("\033[34m", end="")
        response_message = ""
        for chunk in stream:
            if chunk.choices:
                next = chunk.choices[0].delta.content
                if next is not None:
                    response_message += next
                    print(next, end='', flush=True)
        print("\033[0m")

        # メッセージに言語モデルからの回答を追加
        user_messages.append({"role": "assistant", "content": response_message})
        log(f"Stored answer. {response_message=} {user_messages=}")

    print("\n---ご利用ありがとうございました！---")
    log("Finished main program.")


### =======================================
### 以下、ユーティリティクラスとなります
### =======================================


class OpenAIClientGenerator:

    OPEN_API_ENV_NAME = "API_KEY"
    
    # この関数を呼び出すことで .env ファイルもしくは環境変数から OpenAI API をロードして
    # OpenAI の client インスタンスを返します
    # api_key_validation      ... True なら簡易的なキーのチェックを行います
    # display_debug_messages  ... True ならデバッグメッセージを表示します
    @classmethod
    def generate(cls, api_key_validation=True, display_debug_messages=False):
        return OpenAI(api_key=cls.__load_api_key(api_key_validation, display_debug_messages))


    # この関数で OpenAI API のキーを読み取ります
    # 1. 実行パスの ../.env が存在するならば API_KEY を環境変数としてロードして読み取る
    # 2. 環境変数の API_KEY をロードして読み取る
    # 3. API_KEY の中身の値が絶対ファイルパスならばその中身をそのままロードする
    #    そうでないならば中身をそのまま API キーとして使う
    # 4. キーの簡易的なフォーマットチェック
    @classmethod
    def __load_api_key(cls, api_key_validation, display_debug_messages):        
        api_key = None

        log = cls.get_logger(display_debug_messages)

        # 1. ../.env を読み取る
        env_file_path = Path().resolve().parent.resolve() / ".env"
        if env_file_path.is_file():
            try:
                from dotenv import load_dotenv
                load_dotenv(env_file_path)
                log(f"Loaded .env environment variables correctly. env={env_file_path.resolve()}")
            except:
                log("=== EXCEPTION ============")
                log(traceback.format_exc())
                log("==========================")
                log("Found .env file but failed to load dotenv file! Please install python-dotenv module.")
        
        # 2. API_KEY を読み取る
        api_key = os.environ.get(cls.OPEN_API_ENV_NAME, None)

        # 3. API_KEY の中身チェック
        api_file_path = Path(api_key).expanduser()
        if (api_file_path.is_absolute() and api_file_path.is_file()):
            log("Found OpenAI API key file.")
            with open(api_file_path, "r") as f:
                api_key = f.read().strip()

        # 4. キーの簡易チェック
        if api_key_validation:
            if re.match(r"^sk\-.*$", api_key) is None:
                raise Exception("Failed to load api key!")

        return api_key

    
    # 簡易 logging クラス代わり、ロガーを入手するための関数
    @classmethod
    def get_logger(cls, display_debug_messages):
        return cls.log_print if display_debug_messages else (lambda *arg, **kwargs: None)


    # 簡易 logging クラス代わり、ロガーで印字する関数
    @staticmethod
    def log_print(msg, *args, **kwargs):
        formated_msg = f"[LOG] {msg}"
        print(formated_msg, *args, **kwargs)


if __name__ == "__main__":
    main()